This exercise made by Pavel Jankiewicz.

In [6]:
from utils import css_from_file
css_from_file('style/style.css')

In [7]:
import re
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import (
    CountVectorizer, TfidfVectorizer, HashingVectorizer
)

import warnings
warnings.filterwarnings("ignore")

Read csv file into a dataframe

In [8]:
df = pd.read_csv("data/ebaytitles.csv")
df = df.sample(frac=0.1) # delete this line if you are brave and have many GBs of RAM
df.head()

,title,category_name
896197,Tube Lined Victorian Style Green Garland Firep...,"Home, Furniture & DIY"
349617,Cotton Round Funky Retro Bathroom Mat Blue Bla...,"Home, Furniture & DIY"
225735,brand new with tags mens DOCKERS the oxford sh...,"Clothes, Shoes & Accessories"
374192,Lavender - Ellagance Series Snow - 20 Seeds,Garden & Patio
81984,2m PS2 Keyboard / Mouse Extension Cable 2 metr...,Computers/Tablets & Networking


Print out unique values of a column

In [10]:
df.category_name.unique()

array(['Home, Furniture & DIY', 'Clothes, Shoes & Accessories',
       'Garden & Patio', 'Computers/Tablets & Networking',
       'Health & Beauty', 'Mobile Phones & Communication',
       'Sound & Vision', 'Vehicle Parts & Accessories',
       'Business, Office & Industrial', 'Sporting Goods',
       'Jewellery & Watches', 'Music', 'Collectibles', 'Art',
       'Toys & Games', 'Dolls & Bears', 'Crafts', 'DVDs, Films & TV',
       'Books, Comics & Magazines', 'Musical Instruments & Gear',
       'Coins & Paper Money', 'Pottery, Porcelain & Glass', 'Baby',
       'Cameras & Photography', 'Everything Else', 'Pet Supplies',
       'Sports Memorabilia', 'Video Games & Consoles',
       'Consumer Electronics', 'Antiques', 'Wholesale & Job Lots',
       'Stamps', 'Cell Phones & Accessories', 'Travel',
       'Entertainment Memorabilia', 'Holidays & Travel', 'Events Tickets'],
      dtype=object)

Split the data into train and test observations - there is a column

In [11]:
X = df.title.values
y = df.category_name.values

X_tr, X_te, y_tr, y_te = train_test_split(X, 
                                          y,
                                          test_size=0.1,
                                          random_state=0)

Exercise 
------------------

1. Count how many titles are in each category (```pandas.DataFrame.groupby```). Print out most common at the top

In [ ]:
##########################
# put your solution here #
##########################

<a>Double click to show the solution</a>
<div class='spoiler'>

frequencies = df.groupby("category_name")["title"].count()
frequencies.sort_values(inplace=True,ascending=False)
print(frequencies)

# or faster

df.category_name.value_counts()

</div>

Bag of words
--------------------

Different types of vectorizers:

<ul>
<li>```sklearn.feature_extraction.text.CountVectorizer``` - Counts the number of times a word appears in the text</li>
<li>```sklearn.feature_extraction.text.TfidfVectorizer``` - Weighs the words according to the importance of the word in the context of whole collection. Is the word ```the``` important if it appears in all documents?</li>
<li>```sklearn.feature_extraction.text.HashingVectorizer``` - Useful when you don't know the vocabulary upfront. Feature number is calculated as ```hash(token) % vocabulary_size```.</li>
</ul>

Exercise
-------------------
1. Use ```CountVectorizer``` / ```TfidfVectorizer``` to fit the collection of documents
2. How many unique tokens are there in text? Print some examples (ie first few hundred).
3. What methods you can use to reduce this number? 
   - Check out and experiment with the arguments: ```ngram_range```, ```min_df```. How the vocabulary size changes with each change?
   - What would you replace / delete from the text?
4. Write a custom function `clean_text` that accepts a text as input and transforms it (remove/hash numbers, delete short/long words etc.)
5. (Extra points) When would you use ```HashingVectorizer```?

In [ ]:
##########################
# put your solution here #
##########################

<a>Double click to show the solution</a>
<div class='spoiler'>
from sklearn.feature_extraction.text import CountVectorizer
import re

def clean_text(t):
    t = t.lower()
    t = re.sub("[^A-Za-z0-9]"," ",t)
    t = re.sub("[0-9]+","#",t)
    return t

vectorizers = [
     ("vanilla",
          CountVectorizer())
    ,("preprocessing",
          CountVectorizer(preprocessor=clean_text))
    ,("preprocessing + min_df=10",
          CountVectorizer(preprocessor=clean_text,
                          min_df=10))
]

for vect_name, vect in vectorizers:
    print(vect_name)
    vect.fit(X_tr)
    
    print(list(vect.get_feature_names())[:10])
    print(len(vect.get_feature_names()))
</div>

Stemming
------------------

Linguistic normalization in which variant forms are reduced to a common form

    connection
    connections
    connective     --->   connect
    connected
    connecting
    
Usage:

    import snowballstemmer

    stemmer = snowballstemmer.stemmer('english')
    print(stemmer.stemWords("We are the world".split()))

Putting it into a pipeline
----------------------

Now that we know how to transform text data, let's put it into a pipeline.

1. Create a pipeline with `CountVectorizer`, `StandardScaler` and `SGDClassifier` as your final algorithm
    a) use alternative format for pipeline definition when you name the steps - refer to the documentation how to do this
2. Using ```sklearn.metrics.classification_report``` create a report about your classifier

In [ ]:
##########################
# put your solution here #
##########################

from sklearn.pipeline import Pipeline, make_pipeline()
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV

clf = make_pipeline(CountVectorizer(), SGDClassifier())
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('sgd', SGDClassifier())
])

preds = cross_val_predict(clf, 
                          X_tr, 
                          y_tr, 
                          cv=8, n_jobs=-1, verbose=True)

print(classification_report(y_tr, preds)

clf.fit(X_tr, y_tr)

<a>Double click to show the solution</a>
<div class='spoiler'>

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV

clf = Pipeline([('vect', CountVectorizer(min_df=10, preprocessor=clean_text)),
                ('scaling', StandardScaler(with_mean=False)),
                ('clf', SGDClassifier())])

preds = cross_val_predict(clf, 
                          X_tr, 
                          y_tr, 
                          cv=8, n_jobs=-1, verbose=True)

print(classification_report(y_tr, preds))


</div>


Grid search
--------------------------

Scikit-learn has `GridSearchCV` and `RandomizedSearchCV`. Both have the same functionality and can be used to find good parameters for the models. What is great about both these classes that they are both transformers - they return an estimator so you can chain them and put in your pipeline.

**GridSearchCV** - you specify the exact values of the parameters you want to test
**RandomizedSearchCV** - you specify ranges of parameters

Exercise
----------------------

1. Use `GridSearchCV` or `RandomizedSearchCV` to find the best parameters for the models. Check at least 2 parameters.

2. Inspect the attribute `cv_results_` after fitting. It gives a nice representation of the learning.

In [18]:
##########################
# put your solution here #
##########################

GridSearchCV(pipeline, {'sgd__alpha':[0.1,0.01,0.001]
                        'vect__min_df':[1,5,10,25],
                        'vect__ngram_range'[(1,1),(1,2),(1,3)]})

<a>Double click to show the solution</a>
<div class='spoiler'>

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

print("Grid search")
print()

params = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)],
          'vect__analyzer': ["word","char"],
          'vect__binary': [True, False]}

grid_clf = GridSearchCV(clf, params, n_jobs=1, verbose=True)
grid_clf.fit(X_tr, y_tr)

best_params = sorted(grid_clf.grid_scores_, key=lambda x: -x[1])


for params, score, _ in best_params:
    print(score, params) 
    
print("Randomized search")
print()
    
params = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)],
          'vect__analyzer': ["word","char"],
          'model__lr__dimensions': [100, 200]}

grid_clf = RandomizedSearchCV(clf, params, n_jobs=1, verbose=True, n_iter=8)
grid_clf.fit(np.array(X_tr[:10000]), y_tr[:10000])

best_params = sorted(grid_clf.grid_scores_, key=lambda x: -x[1])

for params, score, _ in best_params:
    print(score, params)

</div>


Useful materials

1. http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html